<a href="https://colab.research.google.com/github/Shai670/Residential_consumption_modeling/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22battery04_09_ipynb%22%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import clear_output

import matplotlib.pyplot as plt

In [ ]:
!git clone https://github.com/Shai670/Residential_consumption_modeling
clear_output()

In [ ]:
pv = pd.read_csv('/content/Residential_consumption_modeling/DATA/Kemerovo_2022_solcast_3.csv', index_col='time')
pv.index = pd.date_range(start='2022-01-01 07:00:00', freq='5MIN', periods=len(pv))
pv

In [ ]:
load = pd.read_csv("/content/Residential_consumption_modeling/DATA/load.csv", index_col=0)
load.index = pd.to_datetime(load.index)
load

In [ ]:
class battery:


  def __init__(self, Pnom, capacity):
    self.Pnom = Pnom # номинальная мощность батареи
    self.capacity = capacity # ёмкость Вт*ч

    self.SOC_min = .20
    self.SOC_max = .85
    self.SOC = self.SOC_min

    #self.capacity_fact = self.SOC/100 * self.capacity

  def working (self, Pgen, Pload):
     Pbat = 0

     if Pgen > Pload:
       if self.SOC <= self.SOC_max:
         if (Pgen - Pload) >= self.Pnom:
           Pbat = self.Pnom
         elif (Pgen - Pload) < self.Pnom:
           Pbat = Pgen - Pload

     else:
       if self.SOC >= self.SOC_min:
         if Pload - Pgen >= self.Pnom:
           Pbat = -self.Pnom
         elif Pload - Pgen < self.Pnom:
           Pbat = -(Pload-Pgen)

     self.SOC += 5/60 * Pbat / self.capacity

     # self.capacity_fact += 1/60 * Pbat
     # self.SOC = self.capacity_fact / self.capacity * 100

     return Pbat


In [ ]:
dataset = pd.merge(load, pv, left_index=True, right_index=True, how='outer').dropna()
dataset.loc[dataset['Solar'] < 0, 'Solar'] = 0
dataset['Solar'] *= 1e-3
dataset

In [ ]:
bat = battery(Pnom=1, capacity=5)

start = '2022-07-01'
end = '2022-07-05'

load_data = dataset['load'].loc[start : end]
gen_data  = dataset['Solar'].loc[start : end]

Pbat = []; soc = []; Pgrid = []
for load, gen in zip(load_data, gen_data):
  bat_power = bat.working(Pgen=gen, Pload=load)
  grid_power = load + bat_power - gen

  if grid_power < 0: grid_power = 0

  Pgrid.append(grid_power)
  Pbat.append(bat_power)
  soc.append(bat.SOC)

Pbat = pd.DataFrame({'bat' : Pbat}, index=load_data.index)
Pgrid = pd.DataFrame({'grid' : Pgrid}, index=load_data.index)
soc = pd.DataFrame({'soc' : soc}, index=load_data.index)

plt.figure(figsize=(12,6))
plt.plot(load_data, label='Нагрузка')
plt.plot(gen_data, label='Генерация')
plt.plot(Pbat, label='Батарея')

plt.legend()
plt.show()

plt.figure(figsize=(12,6))
plt.plot(soc, label='Нагрузка')
plt.show()

In [ ]:
def solar_battery_calc(Pnom, capacity):
    bat = battery(Pnom=Pnom, capacity=capacity)

    load_data = dataset['load'].copy()
    gen_data  = dataset['Solar'].copy()

    Pbat = []; soc = []; Pgrid = []
    for load, gen in zip(load_data, gen_data):
        bat_power = bat.working(Pgen=gen, Pload=load)
        grid_power = load + bat_power - gen

        if grid_power < 0: grid_power = 0

        Pgrid.append(grid_power)
        Pbat.append(bat_power)
        soc.append(bat.SOC)

    output = pd.DataFrame(data={"Pbat" : Pbat, "SOC" : soc, "Pgrid" : Pgrid}, index=gen_data.index)

    return output

In [ ]:
dataset['load'].mean() * 365 * 24

In [ ]:
output = solar_battery_calc(Pnom=0, capacity=5)
output.mean() * 365 * 24

In [ ]:
output = solar_battery_calc(Pnom=3, capacity=10)
output.sum()

In [ ]:
output = solar_battery_calc(Pnom=5, capacity=10)
output.sum()

In [ ]:
output = solar_battery_calc(Pnom=7.2, capacity=20)
output.sum()

In [ ]:
result = pd.DataFrame()

i = 0
for p in np.arange(0.1, 5, .1):
    for c in np.arange(5, 20, .1):
        output = solar_battery_calc(Pnom=p, capacity=c)

        result.loc[i, "Мощность, кВт"] = p.round(0)
        result.loc[i, "Ёмкость, кВт*ч"] = c.round(0)
        result.loc[i, "W, руб."] = output["Pgrid"].sum().round(1)*3.01*8760

        i += 1
plt.plot(p, label='Мощность, кВт')
plt.plot(c, label='Ёмкость, кВт*ч')
plt.plot(Pgrid, label='W, руб.')

result



In [ ]:

result_pivot = pd.pivot_table(
    result, index="Мощность, кВт", columns="Ёмкость, кВт*ч", values="W, руб."
)
result_pivot

In [ ]:
import seaborn as sns

sns.heatmap(result_pivot)